In [1]:
import pandas as pd
import numpy as np

In [2]:
clip_emd = pd.read_pickle('../clip-training/tweet_data_all_emb.pkl')

In [29]:
df_train = pd.read_pickle('../clip-training/train.pkl')
df_val = pd.read_pickle('../clip-training/eval.pkl')
df_test = pd.read_pickle('../clip-training/test_data.pkl')

df_select = pd.concat([df_train, df_val])
df_full = pd.concat([df_train, df_val, df_test])

In [4]:
train_emd = clip_emd[clip_emd['tweet_id'].isin(df_select['tweet_id'])]

In [30]:
full_emd = clip_emd[clip_emd['tweet_id'].isin(df_full['tweet_id'])]

In [5]:
# train_emd

In [31]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Construction of Graph

In [32]:
from sentence_transformers import util
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from tqdm import tqdm

In [33]:
import networkx as nx
import igraph as ig

In [34]:
# similar matrix as the graph adjacency matrix
# cos_mat = util.cos_sim(train_emd['clip_imgs_sum_text_embeddings'].tolist(),train_emd['clip_imgs_sum_text_embeddings'].tolist()) 
# cos_mat = util.cos_sim(train_emd['clip_imgs_cat_text_embeddings'].tolist(),train_emd['clip_imgs_cat_text_embeddings'].tolist())
cos_mat = util.cos_sim(full_emd['clip_imgs_sum_text_embeddings'].tolist(),full_emd['clip_imgs_sum_text_embeddings'].tolist()) 


In [35]:
cos_copy = cos_mat.detach().cpu().numpy()

In [36]:
cos_copy.shape

(5000, 5000)

In [37]:
# choose the threshold
cos_copy[cos_copy < 0.85] = 0

In [38]:
G = nx.from_numpy_array(cos_copy)

In [14]:
# nx.write_gpickle(G, "graphSum85.gpickle")
# G = nx.read_gpickle("graph85.gpickle")

In [39]:
g = ig.Graph.from_networkx(G)

# Leiden Clustering

In [40]:
import leidenalg as la

In [41]:
communities = la.find_partition(g, la.ModularityVertexPartition, n_iterations=-1)
optimizer = la.Optimiser()        
diff = 1
while diff > 0:
    diff = optimizer.optimise_partition(communities,n_iterations=-1)

In [42]:
len(communities)

232

In [43]:
dict_stats = {}
k = 0
for i in range(len(communities)):
    if not dict_stats.get(len(communities[i])):
        dict_stats[len(communities[i])] =0
    dict_stats[len(communities[i])] +=1

In [44]:
dict_stats

{2760: 1, 1802: 1, 175: 1, 7: 1, 3: 8, 2: 12, 1: 208}

In [21]:
dict_com = {}
k = 0
for i in range(len(communities)):
    if (len(communities[i])>=10):
        dict_com[k] = train_emd.loc[train_emd.index[communities[i]]]
        k += 1

### leiden graph

In [23]:
# communities[0][0].degree

In [45]:
def get_degree(cm_id, node_id):
    return len(set(communities[cm_id]).intersection(communities.graph.neighborhood(communities[cm_id][node_id])))

def cut_edges(cm_id, node_id):
    
    global_id = communities[cm_id][node_id]
    neighbors_within_cluster = set(communities[cm_id]).intersection(communities.graph.neighborhood(communities[cm_id][node_id]))
    edges, values = np.argsort(cos_copy[global_id][list(neighbors_within_cluster)])[-100:][::-1], np.sort(cos_copy[global_id][list(neighbors_within_cluster)])[-100:][::-1]
    edges = [e for e, v in zip(edges, values) if v>0 and e!=global_id]
    
    return edges

def get_more_edges(cm_id, node_id,similarity_matrix):
    neighbors_within_cluster = set(communities[cm_id]).intersection(communities.graph.neighborhood(communities[cm_id][node_id]))
    global_id = communities[cm_id][node_id]
    edges, values = np.argsort(similarity_matrix[global_id])[-10:][::-1], np.sort(similarity_matrix[global_id])[-10:][::-1]
    edges = [e for e, v in zip(edges, values) if e!=global_id]
    return edges
def get_edges(cm_id, node_id):
    global_id = communities[cm_id][node_id]
    edges = [e for e in communities.graph.neighborhood(communities[cm_id][node_id]) if e!=global_id]
    return edges

In [26]:
# neighbors_within_cluster

In [46]:
gnn_graph = nx.Graph()

# similar matrix as the graph adjacency matrix
# similarity_matrix = util.cos_sim(train_emd['clip_imgs_sum_text_embeddings'].tolist(),train_emd['clip_imgs_sum_text_embeddings'].tolist()).detach().cpu().numpy()
similarity_matrix = util.cos_sim(full_emd['clip_imgs_sum_text_embeddings'].tolist(),full_emd['clip_imgs_sum_text_embeddings'].tolist()).detach().cpu().numpy()

# For each community (i.e., cluster)
for cm_id , communitie in tqdm(enumerate(communities)):
    # for each node within the cluster
    for node_id, global_id in enumerate(communitie):
        
        # Check if the node degree is higher than 100
        if get_degree(cm_id, node_id) > 100:
            # cut edges
            edges = cut_edges(cm_id, node_id)
            for e in edges:
                # Fix to tweetID
#                 gnn_graph.add_edge(train_emd.iloc[global_id]['tweet_id'], train_emd.iloc[e]['tweet_id'])
                gnn_graph.add_edge(full_emd.iloc[global_id]['tweet_id'], full_emd.iloc[e]['tweet_id'])

            
        # Check if the node has at least 10 edges    
        elif get_degree(cm_id, node_id) < 10:
            # include an edge to the most similar nodes
            edges = get_more_edges(cm_id, node_id, similarity_matrix)
            
            for e in edges:
                # Fix to tweetID
                gnn_graph.add_edge(full_emd.iloc[global_id]['tweet_id'], full_emd.iloc[e]['tweet_id'])
        else:
              for e in get_edges(cm_id, node_id):
                    # Fix to tweetID
                    gnn_graph.add_edge(full_emd.iloc[global_id]['tweet_id'], full_emd.iloc[e]['tweet_id'])

232it [02:09,  1.79it/s]


In [47]:
nx.write_gpickle(gnn_graph, "graphLeidenFullSum.gpickle")

## Select representative nodes based on centrality measures

In [21]:
# create a new graph for each cluster
def build_graph(df_cluster, th = 0.85):
    cos_sub = util.cos_sim(df_cluster['clip_imgs_sum_text_embeddings'].tolist(),df_cluster['clip_imgs_sum_text_embeddings'].tolist()).detach().cpu().numpy()
#     cos_sub = util.cos_sim(df_cluster['clip_imgs_cat_text_embeddings'].tolist(),df_cluster['clip_imgs_cat_text_embeddings'].tolist()).detach().cpu().numpy()
    cos_sub[cos_sub < th] = 0
    G = nx.from_numpy_array(cos_sub)
    g = ig.Graph.from_networkx(G)
    return g

### Random Sampling

In [22]:
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].sample(n=len(dict_com[k]), random_state=1))
    cid.append([k]*len(dict_com[k]))

In [23]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('LeidenRandomSum.csv')

### Degree

In [24]:
def degree_centrality(g):
    degree_inx = np.argsort(g.degree())[::-1]
    return degree_inx

In [25]:
## Degree centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[degree_centrality(g)])
    cid.append([k]*len(dict_com[k]))

In [26]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('LeidenDegreeSum.csv')

### Pagerank

In [27]:
def pagerank_centrality(g):
    pagerank_inx = np.argsort(g.pagerank())[::-1]
    return pagerank_inx

In [28]:
## PageRank centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[pagerank_centrality(g)])
    cid.append([k]*len(dict_com[k]))

In [29]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('LeidenPageRankSum.csv')

### Betweenness

In [30]:
def betweenness_centrality(g):
    between_inx = np.argsort(g.betweenness())[::-1]
    return between_inx

In [31]:
## Betweenness centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[betweenness_centrality(g)])
    cid.append([k]*len(dict_com[k]))

In [32]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('LeidenBetweennessSum.csv')

### Closeness

In [33]:
def closeness_centrality(g):
    closeness_inx = np.argsort(g.closeness())[::-1]
    return closeness_inx

In [34]:
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[closeness_centrality(g)])
    cid.append([k]*len(dict_com[k]))

In [35]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('LeidenClosenessSum.csv')

### Eigenvector

In [36]:
def eigenvector_centrality(g):
    eigenvector_inx = np.argsort(g.eigenvector_centrality())[::-1]
    return eigenvector_inx

In [37]:
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[eigenvector_centrality(g)])
    cid.append([k]*len(dict_com[k]))

In [38]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('LeidenEigenSum.csv')

### Multi-Centrality Index

=======

In [39]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import sys
sys.path.insert(0,"/fake-news-qa/7Set-AL/MultiCentrality/")
import MultiCentralityIndex as MCI

In [40]:
mc = MCI.MCI()

In [41]:
centralities = ['Degree', 'Pagerank', 'Eigenvector', 'Closeness', 'Betweenness', 'StructuralHoles']

In [42]:
sc = MinMaxScaler()
def mci_centrality(g, df):
    df_mci = pd.DataFrame()
    df_mci['tweet_id'] = df['tweet_id']
    df_mci['MCI'] = np.zeros((len(df),1))
    df_mci['Degree'] = sc.fit_transform(np.asarray(g.degree()).reshape(-1,1))
    df_mci['Pagerank']=sc.fit_transform(np.asarray(g.pagerank()).reshape(-1,1))
    df_mci['Eigenvector']=sc.fit_transform(np.asarray(g.eigenvector_centrality()).reshape(-1,1))
    df_mci['Closeness']=sc.fit_transform(np.asarray(g.closeness()).reshape(-1,1))
    df_mci['Betweenness']=sc.fit_transform(np.asarray(g.betweenness()).reshape(-1,1))
    df_mci['StructuralHoles']=sc.fit_transform(np.asarray(g.constraint()).reshape(-1,1))
    PC1 = mc.getPC1(df_mci)
    
    for cnt in list(centralities):
        df_mci['MCI'] += df_mci[cnt]*PC1.loc[0,cnt] 
        
    mci_inx = np.argsort(df_mci['MCI'])[::-1]
    return mci_inx

In [43]:
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[mci_centrality(g, dict_com[k])])
    cid.append([k]*len(dict_com[k]))

In [44]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('LeidenMCISum.csv')

# Agglomerative clustering

In [64]:
from sklearn.cluster import AgglomerativeClustering

In [65]:
corpus = train_emd['processed_tweet'].tolist()

In [66]:
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0.15, affinity='precomputed', linkage='average')

In [67]:
clustering_model.fit(1-cos_copy)
cluster_assignment = clustering_model.labels_

In [68]:
clustered_sentences = {}
clustered_ids = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []
        clustered_ids[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])
    clustered_ids[cluster_id].append(sentence_id)

In [69]:
communities = sorted(clustered_ids.items(), key= lambda x: len(x[1]), reverse=True) 

In [70]:
len(communities)

400

In [71]:
dict_com = {}
k = 0
for com in communities:
    if (len(com[1])>=10):
        dict_com[k] = train_emd.loc[train_emd.index[com[1]]]
        k += 1

In [74]:
## Degree centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[degree_centrality(g)[:10]])
    cid.append([k]*10)

In [75]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('AggDegreeSum.csv')

In [76]:
## PageRank centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[pagerank_centrality(g)[:10]])
    cid.append([k]*10)

In [77]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('AggPageRankSum.csv')

In [78]:
## Betweenness centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[betweenness_centrality(g)[:10]])
    cid.append([k]*10)

In [79]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('AggBetweennessSum.csv')

In [80]:
## Closeness centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[closeness_centrality(g)[:10]])
    cid.append([k]*10)

In [81]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('AggClosenessSum.csv')

In [82]:
## Eigenvector centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[eigenvector_centrality(g)[:10]])
    cid.append([k]*10)

In [83]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('AggEigenSum.csv')

In [84]:
## MCI centrality
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].iloc[mci_centrality(g, dict_com[k])[:10]])
    cid.append([k]*10)

In [85]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('AggMCISum.csv')

In [97]:
## Random sampling
df_select = []
cid = []
for k in range(len(dict_com)):
    g = build_graph(dict_com[k])
    df_select.append(dict_com[k].sample(n=10, random_state=1))
    cid.append([k]*10)

In [98]:
df = pd.concat(df_select)
df['cluster_id'] = sum(cid, [])
df[['tweet_id', 'cluster_id']].to_csv('AggRandomSum.csv')